In [59]:
import pandas as pd
import numpy as np
import fuzzywuzzy
import wrds


db = wrds.Connection()

Enter your WRDS username [shuhao]:nosleep3
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [5]:
ibes1 = db.raw_sql("""
                      select ticker, cusip, cname, sdates from ibes.idsum
                      where usfirm=1 and cusip != ''
                      """, date_cols=['sdates'])

# first and last 'start date' for a given cusip
ibes1_fdate = ibes1.groupby(['ticker','cusip'])['sdates'].min().reset_index()
ibes1_ldate = ibes1.groupby(['ticker','cusip'])['sdates'].max().reset_index()
ibes1_fdate = ibes1_fdate.rename(columns={'sdates':'fdate'})
ibes1_ldate = ibes1_ldate.rename(columns={'sdates':'ldate'})

ibes2 = pd.merge(ibes1, ibes1_fdate, on=['ticker','cusip'], how='left')
ibes2 = pd.merge(ibes2, ibes1_ldate, on=['ticker','cusip'], how='left')
ibes2 = ibes2.sort_values(by=['ticker','cusip','sdates'])

# keep only the most recent company name
ibes2 = ibes2[ibes2['sdates']==ibes2['ldate']].drop(['sdates'], axis=1)

In [6]:
###################
# CRSP Part       #
###################
# CRSP: Get all permno-ncusip combinations
crsp1 = db.raw_sql("""
                      select distinct permno, ncusip, comnam, 
                      namedt, nameenddt
                      from crsp.stocknames
                      where ncusip != ''
                      """, date_cols=['namedt','nameenddt'])

# Given a permno-ncusip combo find the ealiest namedt and last nameenddt
crsp1_namedt = crsp1.groupby(['permno','ncusip'])['namedt'].min().reset_index()
crsp1_nameenddt = crsp1.groupby(['permno','ncusip'])['nameenddt'].max().reset_index()
crsp1_dtrange = pd.merge(crsp1_namedt, crsp1_nameenddt, \
                          on = ['permno','ncusip'], how='inner')

crsp1 = crsp1.drop(['namedt'],axis=1).rename(columns={'nameenddt':'enddt'})
crsp2 = pd.merge(crsp1, crsp1_dtrange, on =['permno','ncusip'], how='inner')

# keep only most recent company name
crsp2 = crsp2[crsp2['enddt']==crsp2['nameenddt']]
crsp2=crsp2.drop(['enddt'], axis=1)

In [7]:
###################
# Create Link     #
###################

## Option 1: Link by full cusip ##
from fuzzywuzzy import fuzz

link1_1 = pd.merge(ibes2, crsp2, how='inner', left_on='cusip', right_on='ncusip')
link1_1 = link1_1.sort_values(['ticker','permno','ldate'])

# Keep link with most recent company name
link1_1_tmp = link1_1.groupby(['ticker'])['ldate'].max().reset_index()
link1_2 = pd.merge(link1_1, link1_1_tmp, how='inner', on =['ticker','ldate'])

# Calculate distance of company names
# fuzzywuzzy - 100=highest match 0=lowest match
link1_2['name_similarity']=link1_2\
.apply(lambda row: fuzz.token_set_ratio(row['comnam'], row['cname']), axis=1)

# 10% percentile of the company name similarity
name_similarity_p10 = link1_2.name_similarity.quantile(0.10)

# Assign score for companies matched by full cusip and passing name distance
def score1(row):
    if (row['fdate']<=row['nameenddt']) & (row['ldate']>=row['namedt']) \
    & (row['name_similarity'] >= name_similarity_p10):
        value = 0
    elif (row['fdate']<=row['nameenddt']) & (row['ldate']>=row['namedt']):
        value=1
    elif row['name_similarity'] >= name_similarity_p10:
        value=2
    else:
        value=3
    return value

# assign size portfolio
link1_2['score']=link1_2.apply(score1, axis=1)
link1_2 = link1_2[['ticker','permno','cname','comnam','name_similarity','score']]
link1_2 = link1_2.drop_duplicates()

/Users/shuhao/anaconda2/lib/python2.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
## Option 2: Link unmatched by comp.security table ibtic column ##
nomatch1 = pd.merge(ibes2[['ticker','cname']], link1_2[['permno','ticker']], on='ticker', how='left')
nomatch1 = nomatch1[nomatch1.permno.isnull()]
nomatch1 = nomatch1.drop(['permno'], axis=1).drop_duplicates()

# Use ccm table to add gvkey and ibtic
ccm=db.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """)

# join with compustat security for ibtic
comp_sec = db.raw_sql("""
                        select gvkey, iid, ibtic from comp.security
                        """)

ccmib = pd.merge(ccm, comp_sec, on=['gvkey','iid'], how='inner')
ccmib = ccmib[ccmib['ibtic'].notnull()].drop(['gvkey','iid'],axis=1)
ccmib = ccmib.drop_duplicates().rename(columns={'ibtic':'ticker'})

link2_1 = pd.merge(nomatch1, ccmib, how='inner', on='ticker')
link2_2=link2_1[link2_1['permno'].notnull()]

# add back crsp company names for final quality check
# pick latest comnam from crsp
crsp2_last = crsp2.groupby(['permno'])['nameenddt'].max().reset_index()
crsp3 = pd.merge(crsp2, crsp2_last, on=['permno', 'nameenddt'], how='inner')
link2_3 = pd.merge(link2_2, crsp3[['permno','comnam']], on=['permno'], how='left')

# calculate name similarity
link2_3['name_similarity']=link2_3\
.apply(lambda row: fuzz.token_set_ratio(row['comnam'],row['cname']), axis=1)
name_sim_p10 = link2_3.name_similarity.quantile(0.10)

link2_3['score']=np.where(link2_3['name_similarity']>=name_sim_p10, 0, 1)

In [9]:
# Combine the two sources for final output
iclink= link1_2.append(link2_3)
iclink = iclink[['ticker','permno','cname', 'comnam', 'name_similarity','score']]
iclink['permno']=iclink['permno'].astype(int)
iclink = iclink.drop_duplicates()

In [10]:
print(len(iclink))
print(iclink.head())

18272
  ticker  permno                             cname  \
0   0000   14471                TALMER BANCORP INC   
1   0001   14392                    EP ENERGY CORP   
2   0004   14418  AMERICAN CAPITAL SENIOR FLTG LTD   
3   000R   14378                       CARECOM INC   
4   000V   14423      EIGER BIOPHARMACEUTICALS INC   

                            comnam  name_similarity  score  
0               TALMER BANCORP INC              100      0  
1                  E P ENERGY CORP               97      0  
2  AMERICAN CAPITAL SR FLOATING LT               86      0  
3                     CARE COM INC               96      0  
4     EIGER BIOPHARMACEUTICALS INC              100      0  


### Dowload the link table of CRSP and Compustat

In [70]:
# data = db.raw_sql("select distinct gvkey, linkprim, liid, lpermno from crsp.ccmxpf_lnkhist")

# data = data[['gvkey', 'lpermno']].dropna()
# data = data.drop_duplicates(subset=['gvkey', 'permno'], keep='last')
# len(data)
# # df.sort('C').drop_duplicates(subset=['A', 'B'], take_last=True)
# data.to_csv('/Users/shuhao/Dropbox/Textual Analysis Project/Data/Compustat/Compustat_CRSP_link_table_2.csv')

In [27]:
# df_ibes = db.raw_sql("select ticker, cusip, oftic, cname, anndats, anntims,ireccd from ibes.recddet where usfirm = 1 and cusip != ''")
# df_ibes.to_csv('/Users/shuhao/Dropbox/Textual Analysis Project/Data/IBES/IBES_Recommendation.csv')

In [41]:
# df_shrt_int = pd.read_csv('/Users/shuhao/Dropbox/Textual Analysis Project/Data/Compustat/compustat_short_interest.csv')

# df_shrt_int['date'] = pd.to_datetime(df_shrt_int['datadate'])
# df_shrt_int.reset_index(inplace=True)
# df_shrt_int.sort_values(by=['gvkey', 'date']).head(20)

,index,gvkey,iid,shortint,datadate,tic,cusip,date
6,6,1000,1,200.0,08/13/1976,NaN,NaN,1976-08-13
0,0,1000,1,1000.0,07/15/1977,NaN,NaN,1977-07-15
3,3,1000,1,1000.0,08/15/1977,NaN,NaN,1977-08-15
2,2,1000,1,2400.0,09/15/1977,NaN,NaN,1977-09-15
7,7,1000,1,2400.0,10/14/1977,NaN,NaN,1977-10-14
1,1,1000,1,2000.0,11/15/1977,NaN,NaN,1977-11-15
5,5,1000,1,2000.0,12/15/1977,NaN,NaN,1977-12-15
4,4,1000,1,300.0,06/15/1978,NaN,NaN,1978-06-15
431,431,1004,1,1515.0,06/15/1973,AIR,000361105,1973-06-15
543,543,1004,1,2011.0,10/15/1973,AIR,000361105,1973-10-15


In [55]:
# df_shrt_int.to_csv('/Users/shuhao/Dropbox/Textual Analysis Project/Data/Compustat/compustat_short_interest.csv')
# iclink.to_csv('/Users/shuhao/Dropbox/Textual Analysis Project/Data/CRSP_IBES_link_table.csv')

In [49]:
# merge.to_csv('/Users/shuhao/Dropbox/Textual Analysis Project/Data/IBES/merge_IBES_CRSP_Identifier.csv')